In [7]:
!pip install -qU \
  langchain==0.3.25 \
  langchain-community==0.3.25 \
  langchain-openai==0.3.22 \
  numexpr==2.11.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/learn/generation/langchain/handbook)

# Getting Started with Chains

Chains are the core of LangChain. They are simply a chain of components, executed in a particular order.

The simplest of these chains is the `LLMChain`. It works by taking a user's input, passing in to the first element in the chain — a `PromptTemplate` — to format the input into a particular prompt. The formatted prompt is then passed to the next (and final) element in the chain — a LLM.

Nowadays, chains are mostly built using **L**ang**C**hain **E**xpression **L**anguage (LCEL) rather that using objects like the `LLMChain`. So, when we refer to the `LLMChain` we are references this _past_ object but nowadays this would be implemented via LCEL syntax - which we'll explore soon.

Let's start by importing all the libraries that we'll be using in this example.

To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use throughout the notebook, just input your [OpenAI API key](https://platform.openai.com/api-keys) below.

In [9]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")

In [10]:
from langchain_openai import ChatOpenAI

# initialize the models
llm = ChatOpenAI(
    model_name="gpt-4.1-mini",
    temperature=0.7
)

## Chains and LCEL

Chains in LangChain are now built using the LangChain Expression Language (LCEL), which takes a declarative approach to combining components. Instead of using predefined chain classes, LCEL lets you compose chains using the `|` operator and other composition primitives.

### Types of Chain Composition

1. **Sequential Chains** (`|` operator)
   - Chain components one after another
   - Example: `prompt | llm | output_parser`

2. **Parallel Chains** (`RunnableParallel`)
   - Run multiple operations concurrently
   - Example: Running multiple prompts or retrievers in parallel

3. **Complex Workflows**
   - For more complex scenarios involving branching, cycles, or multiple agents
   - Recommended to use LangGraph instead of LCEL directly

Let's start with a simple example: creating a sequential math chain that can handle calculations...

In [12]:
import numexpr
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# Create a function to handle calculations
def calculate(expression: str) -> str:
    """Calculate using numexpr, with support for basic math operations."""
    try:
        result = float(numexpr.evaluate(expression))
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Create the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful math assistant. When given a math problem, respond ONLY with the mathematical expression that would solve it. For example, if asked 'What is 2 raised to the 3rd power?', respond only with '2**3'."),
    ("user", "{question}")
])

# Wrap our calculation function with RunnableLambda for explicit LCEL pattern
calculate_runnable = RunnableLambda(calculate)

# Create the chain using LCEL with explicit RunnableLambda
math_chain = (
    prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()  # Convert to string
    | calculate_runnable  # Our calculation function wrapped in RunnableLambda
)

# Use the chain with our example
response = math_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)

The result is: 2.4116004626599237


Let's see what is going on here. The chain processes our input through several sequential steps:

1. The prompt template formats our question
2. The LLM converts it to a mathematical expression
3. The StrOutputParser ensures we get a clean string
4. Finally, our calculate function computes the result

But how did the LLM know to return just the mathematical expression? 🤔

**Enter prompts**

The question we send isn't the only input the LLM receives 😉. Look at our prompt template:


In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful math assistant. When given a math problem, respond ONLY with the mathematical expression that would solve it. For example, if asked 'What is 2 raised to the 3rd power?', respond only with '2**3'."),
    ("user", "{question}")
])

The system message explicitly instructs the LLM to return only the mathematical expression. Without this context, the LLM would try to calculate the result itself. Let's test this by trying without the system message:


In [14]:
# Simple prompt without guidance
prompt = ChatPromptTemplate.from_messages([
    ("user", "{question}")
])

basic_chain = (
    prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

response = basic_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)  # The LLM tries to calculate it directly and gets it wrong!


13^0.3432 is approximately equal to 2.732.


This demonstrates the power of prompting in LCEL: by carefully designing our prompts, we can guide the LLM's behavior precisely.

The beauty of LCEL's sequential composition is how clearly we can see each step in the chain:


In [15]:
math_chain = (
    prompt                         # Step 1: Format the input with our system message
    | ChatOpenAI(temperature=0)    # Step 2: Get mathematical expression from LLM
    | StrOutputParser()            # Step 3: Convert to clean string
    | calculate                    # Step 4: Evaluate the expression
)

Each step flows naturally into the next using the `|` operator, making it easy to understand and modify the chain's behavior. This is much more flexible than the old approach of using predefined chain classes - we can easily add, remove, or modify steps as needed!

*_Note: The `calculate` function uses `numexpr` to safely evaluate mathematical expressions without needing a full Python REPL (Read-Eval-Print Loop)._

### Building Complex Chains with LCEL

Let's build a more complex example that shows how to combine different components using LCEL. We'll create a chain that cleans up messy text and then paraphrases it in a specific style.

First, let's create a function to clean up text by removing extra spaces and newlines. In LCEL, we can use regular functions directly in our chain:

In [16]:
import re

def clean_text(text: str) -> str:
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text

Now, let's create our prompt template for the paraphrasing:

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative writing assistant."),
    ("user", """Please paraphrase this text in the style of {style}:

{text}""")
])

Now we can combine everything into a sequential chain using LCEL's `|` operator. The beauty of LCEL is how naturally we can compose these components:


In [18]:
# Create the chain using LCEL
style_chain = (
    {
        "text": lambda x: clean_text(x["text"]),  # Extract and clean the text from input dict
        "style": lambda x: x["style"]  # Extract style from input dict
    }
    | prompt  # Format with our template
    | ChatOpenAI(temperature=0.7)  # Generate creative paraphrase
    | StrOutputParser()  # Convert to string
)

# Our input text with messy spacing
input_text = """
Chains allow us to combine multiple


components together to create a single, coherent application.

For example, we can create a chain that takes user input,       format it with a PromptTemplate,

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by


combining chains with other components.
"""

# Run the chain
response = style_chain.invoke({
    "text": input_text,
    "style": "a 90s rapper"
})
print(response)

Yo, check it - chains in the game,
Bringin' all the components together, ain't it insane?
Craftin' a slick app that's all in sync,
User input to PromptTemplate, then pass it to LLM, that's how we link.
Mix and match, buildin' up them chains,
Complexity on the rise, ain't no reins.
Merge 'em all, blend with other parts,
In the world of creation, we're makin' our marks.


Let's look at how this chain works:

1. The dictionary `{"text": clean_text, "style": lambda x: x}` processes our inputs in parallel using `RunnableParallel`
2. The `|` operator connects each component, showing the clear flow of data
3. Each step in the chain serves a specific purpose and is easily modifiable
4. The components work together seamlessly to process and transform the text

This demonstrates how LCEL lets us compose simple components into powerful chains while keeping the code readable and maintainable. Whether you're processing text, generating content, or building complex workflows, LCEL's composition primitives make it easy to build exactly what you need! 🔥

### Using RunnableParallel and RunnablePassthrough

Let's explore how to use `RunnableParallel` for running multiple operations concurrently and `RunnablePassthrough` for passing data through unchanged:

In [19]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Create two different analysis prompts
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analysis expert. Analyze the emotional tone."),
    ("user", "What's the sentiment of: {text}")
])

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a summarization expert."),
    ("user", "Summarize in one sentence: {text}")
])

# Use RunnableParallel to run both analyses simultaneously
analysis_chain = RunnableParallel(
    {
        "sentiment": sentiment_prompt | ChatOpenAI(temperature=0) | StrOutputParser(),
        "summary": summary_prompt | ChatOpenAI(temperature=0) | StrOutputParser(),
        "original": RunnablePassthrough()  # Pass through the original input
    }
)

# Test it
sample_text = {"text": "The product exceeded my expectations. Great quality!"}
results = analysis_chain.invoke(sample_text)

print("Sentiment:", results["sentiment"])
print("Summary:", results["summary"])
print("Original:", results["original"]["text"])

Sentiment: The sentiment of the statement is positive. The use of words like "exceeded my expectations" and "great quality" indicates a high level of satisfaction and positivity towards the product.
Summary: The product surpassed expectations with its excellent quality.
Original: The product exceeded my expectations. Great quality!


### Batch Processing with LCEL

LCEL chains support efficient batch processing using the `.batch()` method:

In [20]:
# Create a simple question-answering chain
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer concisely."),
    ("user", "{question}")
])

qa_chain = qa_prompt | ChatOpenAI(temperature=0) | StrOutputParser()

# Batch of questions
questions = [
    {"question": "What is the capital of France?"},
    {"question": "Who wrote Romeo and Juliet?"},
    {"question": "What is the speed of light?"}
]

# Process all questions in batch
answers = qa_chain.batch(questions)

# Display results
for q, a in zip(questions, answers):
    print(f"Q: {q['question']}")
    print(f"A: {a}\n")

Q: What is the capital of France?
A: The capital of France is Paris.

Q: Who wrote Romeo and Juliet?
A: William Shakespeare wrote Romeo and Juliet.

Q: What is the speed of light?
A: The speed of light in a vacuum is approximately 299,792 kilometers per second (or about 186,282 miles per second).



That's it for this example on chains with LCEL.

---